# TAREA 2

## Datos

### Lectura de datos

Como se denotó en la tarea anterior los datos a utilizar son los correspondientes al Inventario Nacional de Emisiones de Gases y Compuestos de Efecto Invernadero del INECC que se puede encontrar en la siguiente liga: https://datos.gob.mx/busca/dataset/inventario-nacional-de-emisiones-de-gases-y-compuestos-de-efecto-invernadero-inegycei.

De la liga se usan un total de 2 archivos:
* Inventario Nacional de Emisiones de Gases y Compuestos de Efecto Invernadero (INEGYCEI) 1990-2019
* Inventario Nacional de Emisiones de Gases y Compuestos de Efecto Invernadero INEGYCEI 2020-2021

Ambos archivos de Excel tienen formatos que generan conflictos al momento de importarlos por lo que se usó la pestaña de *Automatización* y la función de *Grabar acciones* de Excel para generar un script para copiar solo los datos de las hojas a hojas de trabajo nuevas:

``````
function main(workbook: ExcelScript.Workbook) {
	// Add a new worksheet
	let hoja3 = workbook.addWorksheet();
	let selectedSheet = workbook.getActiveWorksheet();
	// Paste to range A1 on hoja3 from range A2:AF190 on selectedSheet
	hoja3.getRange("A1").copyFrom(selectedSheet.getRange("A2:AF190"), ExcelScript.RangeCopyType.values, false, false);
	// Delete range 1:1 on hoja3
	hoja3.getRange("1:1").delete(ExcelScript.DeleteShiftDirection.up);
}
``````
Una vez generadas las nuevas hojas de datos se les cambia el nombre y se unen todas en un solo archivo al que se le llamó ***solo_valores_1990_2021***.

Se importan las librerías que se necesitarán

In [1]:
import pandas as pd
import numpy as np
import math

Leer el archivo y guardar las diferentes hojas en un dataframe distinto

In [2]:
archivo = 'C:/Users/patty/Documents/Maestria Base de Datos/Aprendizaje_Automatico_priv/solo_valores_1990_2021.xlsx'

df = {}
for i in range(1990,2022):
    j = str(i)+'.2'
    df[i] = pd.read_excel(archivo, sheet_name= j)

Para esto se crea un diccionario **df** vacío. Posteriormente se inicia un ciclo para asignar una hoja distinta a cada elemento del diccionario.

Se ha de notar que el ciclo se hizo en el rango de 1990 a 2022 para después convertir dicho número a cadena y agregar *'.2'* ya que esos son los nombres que tienen las hojas en el libro de Excel.

Una vez terminado podemos mandar a llamar un dataframe para revisar los datos:

In [3]:
df[1990]

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,0,0,0,0,0,36.517125,467413.123797,466720.011075,NaN,77.5572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Aviación internacional,2052.937,0.396,14.985,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2068.318,NaN,NaN,NaN
177,Marítimo internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Emisiones de CO2 por quema de biomasa,35955.863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35955.863,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos ver que se cuenta con 32 columnas de las cuales solo la primera columna es del tipo *string* mientras que el resto es del tipo *float*.

### Limpieza de datos

#### Comparación de dataframes

Esto se realiza ya que los dataframes provienen de distintos archivos

Revisamos que todos los dataframes tengan el mismo tamaño con la ayuda de la función *shape*:

In [4]:
c = True

for i in range(1990,2021):
    if df[i].shape != df[i+1].shape:
        c = False
c

False

Al hacer una pequeño cambio al ciclo podemos saber a partir de que año cambia el tamaño de los data frame:

In [5]:
c = True

for i in range(1990,2021):
    if df[i].shape != df[i+1].shape:
        c = False
        y=i
        break
y

2019

Comparamos los tamaños de cada dataframe para saber donde varían:

In [6]:
df[2019].shape

(181, 32)

In [7]:
df[2020].shape

(187, 32)

In [8]:
df[2021].shape

(187, 32)

Podemos ver que la diferencia es en el número de filas y no en las columnas.

Para poder ver las diferencias en las columnas podemos crear un diccionario con la primera columna de cada dataframe para comparar donde son diferentes:

In [9]:
m= {}

for i in range(0,181):
   m[i]= [df[2019].iloc[i,0] , df[2020].iloc[i,0]]

for i in range(181,187):
   m[i]=['-',df[2020].iloc[i,0]]
m


{0: [nan, nan],
 1: [nan, nan],
 2: ['  Potencial de calentamiento', '  Potencial de calentamiento'],
 3: [nan, nan],
 4: ['EMISIONES NETAS (Gg de CO2e)', 'EMISIONES NETAS (Gg de CO2e)'],
 5: ['[1] Energía', '[1] Energía'],
 6: ['[1A] Actividades de quema del combustible',
  '[1A] Actividades de quema del combustible'],
 7: ['[1A1] Industrias de la energía', '[1A1] Industrias de la energía'],
 8: ['[1A1a] Actividad principal producción de electricidad y calor',
  '[1A1a] Actividad principal producción de electricidad y calor'],
 9: ['[1A1b] Refinación del petróleo', '[1A1b] Refinación del petróleo'],
 10: ['[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía',
  '[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía'],
 11: ['[1A2] Industrias manufactura y de la construcción',
  '     1A1ci Fabricación de combustibles sólidos (coque de carbón)'],
 12: ['[1A2a] Hierro y acero', '1A1cii Otras Industrias de la energía  '],
 13: ['[1A2b] Metale

Para poder mejorar la visualización convertimos el diccionario en un dataframe:

In [10]:
mm = pd.DataFrame.from_dict(m, orient= 'index')
mm

,0,1
0,NaN,NaN
1,NaN,NaN
2,Potencial de calentamiento,Potencial de calentamiento
3,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),EMISIONES NETAS (Gg de CO2e)
...,...,...
182,-,[4D] Tratamiento y eliminación de aguas residu...
183,-,[4D1] Tratamiento y eliminación de aguas resid...
184,-,[4D2] Tratamiento y eliminación de aguas resid...
185,-,[4E] Otros


Imprimimos las primeras 20 y ultimas filas para intentar visualizar las diferencias:

In [11]:
mm.iloc[0:20,:] #primeros 20

,0,1
0,NaN,NaN
1,NaN,NaN
2,Potencial de calentamiento,Potencial de calentamiento
3,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),EMISIONES NETAS (Gg de CO2e)
5,[1] Energía,[1] Energía
6,[1A] Actividades de quema del combustible,[1A] Actividades de quema del combustible
7,[1A1] Industrias de la energía,[1A1] Industrias de la energía
8,[1A1a] Actividad principal producción de elect...,[1A1a] Actividad principal producción de elect...
9,[1A1b] Refinación del petróleo,[1A1b] Refinación del petróleo


In [12]:
mm.iloc[166:187,:] #ultimos 20

,0,1
166,[4C] Incineración y quema a cielo abierto de ...,[3C4] Emisiones directas de los N2O de lo...
167,[4C1] Incineración de residuos peligrosos indu...,[3C5] Emisiones indirectas de los N2O de ...
168,[4C2] Quema a cielo abierto de residuos sólidos,[3C6] Emisiones indirectas de los N2O de ...
169,[4D] Tratamiento y eliminación de aguas residu...,[3C7] Cultivo del arroz
170,[4D1] Tratamiento y eliminación de aguas resid...,[3D] Otros
171,[4D2] Tratamiento y eliminación de aguas resid...,[3D1] Productos de madera recolectada
172,[4E] Otros,[3D2] Otros (especificar)
173,EMISIONES NETAS (Gg de CO2e),[4] Residuos
174,NaN,[4A] Eliminación de residuos sólidos
175,Bunkers,[4A1] Sitios gestionados de eliminación de res...


Podemos ver que las filas varian ya que las bases de datos de 2020 y 2021 tiene un desglose adicional al de los años previos y aunado a eso se tienen más filas al final de las tablas en los años de 1990.

Para saber cuáles son las filas de desglose adicional en los años del 2020 y 2021 podemos usar un ciclo para obtener una lista de las filas:

In [13]:
demas = []
j=0
i=0
while (i<= 181 and j<=186):
    if type(df[2019].iloc[i,0]) != str:
        i=i+1
    if type(df[2020].iloc[j,0]) != str:
        j=j+1
        continue
    if df[2019].iloc[i,0].strip() == df[2020].iloc[j,0].strip():
        j= j+1
        i=i+1
    else:
        demas.append(j)
        j=j+1
         
demas

[11, 12, 17, 18, 19, 22, 23, 24, 25, 26, 35, 36, 37]

Lo que el ciclo hace es revisar la fila *i* para el dataframe del 2019 y la fila *j* del dataframe del 2020.

Primero valida que ambas sean del tipo *string* y de no ser así se pasa a la siguiente fila.

Si al comparar se encuentra que son iguales se pasa a la siguiente fila incrementando el valor de *i* y *j* simultaneamente.
Si se encuentra que no son iguales se agrega *j* a una lista y se pasa a la siguiente fila del dataframe de 2020 incrementando solo *j* para comprobar si esa corresponde a la fila actual del dataframe del 2019.

Además de eso se usa la función *strip* para evitar que haya diferencias debido a espacios vacios en las cadenas.

Una vez que se tiene una lista de las filas podemos quitarlas del dataframe:

In [14]:
temp = df[2020].transpose()

for i in demas:
    n= temp.pop(i)

regresa = temp.transpose()
regresa


,Unnamed: 0,2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,NaN,72.174862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


En esta ocasion se usa la transpuesta del dataframe para hacer posible el uso de la función *pop*.
Se pone la transpuesta en la variable **temp** y se usa un ciclo para remover las columnas(filas) que se marcaron en **demas**. 

Finalmente se vuelve a trasponer el dataframe con la ayuda de **regresa**.

Nótese que se le puede asignar un calor numérico a *pop* ya que al transponer el dataframe el nombre de las columnas son números.


Una vez que se tiene el dataframe corregido puede modificarse con la ayuda de la variable:

In [15]:
df[2020] = regresa
df[2020]

,Unnamed: 0,2020,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,NaN,72.174862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Usamos un método similar para el dataframe de 2021:

In [16]:
temp = df[2021].transpose()

for i in demas:
    n= temp.pop(i)

regresa1 = temp.transpose()
regresa1

,Unnamed: 0,2021,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),456265.876441,198041.61451,36315.62913,1974.597954,1670.82773,5.857883,34.841703,0.275913,8421.011818,...,0.29995,0.000003,0.107289,0.000002,2.96463,2.498574,714047.264077,714047.264077,NaN,66.371687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,25804.882349,5571.51224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31376.394589,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4930.620916,2781.021175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20874.261433,2790.491065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


In [17]:
df[2021]=regresa1
df[2021]

,Unnamed: 0,2021,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),456265.876441,198041.61451,36315.62913,1974.597954,1670.82773,5.857883,34.841703,0.275913,8421.011818,...,0.29995,0.000003,0.107289,0.000002,2.96463,2.498574,714047.264077,714047.264077,NaN,66.371687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,25804.882349,5571.51224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,31376.394589,NaN,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4930.620916,2781.021175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20874.261433,2790.491065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Podemos usar otro método para quitar las filas sobrantes de los otros dataframes:

In [18]:
df[1990].drop(index=range(174,181))

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,0,0,0,0,0,36.517125,467413.123797,466720.011075,NaN,77.5572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN,NaN
170,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN,NaN
171,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN,NaN
172,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Se usa la función *drop* con la indicación de que *index* (filas) tome los valores de 174 a 181.
Esto no cambia el dataframe de raíz ya que al momento de imprimirlo nuevamente, aun se tienen las filas finales:

In [19]:
df[1990]

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,0,0,0,0,0,36.517125,467413.123797,466720.011075,NaN,77.5572
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,Aviación internacional,2052.937,0.396,14.985,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2068.318,NaN,NaN,NaN
177,Marítimo internacional,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Emisiones de CO2 por quema de biomasa,35955.863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35955.863,NaN,NaN,NaN
179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Debido a esto se asignan a una nueva variable. En este caso se usa un diccionario **aux**:

In [20]:
aux = {}

for i in range(1990, 2020):
    aux[i]= df[i].drop(index=range(174,181))

aux[2019]

,Unnamed: 0,2019,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,NaN,65.581718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN,NaN
170,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN,NaN
171,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN,NaN
172,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Ahora con la ayuda del diccionario auxiliar se puede modificar **df**:

In [21]:
for i in range(1990,2020):
    df[i]=aux[i]

In [22]:
df[2019]

,Unnamed: 0,2019,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,8900,1,9540,2,16100,23500,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,NaN,65.581718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN,NaN
170,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN,NaN
171,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN,NaN
172,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN


Comprobamos una vez más que los tamaños sean iguales:

In [23]:
c = True

for i in range(1990,2021):
    if df[i].shape != df[i+1].shape:
        c = False
c

True

#### Corrección de formato de dataframes

##### Corrección de nombres de columnas

Una vez que se han homogenizado las filas se procede a corregir los nombres de las columnas que se encuentran distribuidos en 3 filas:

In [24]:
df[1990].iloc[0:2,:]

,Unnamed: 0,1990,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,\nEMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN,NaN


Podemos ver que las primera fila, la fila de columnas es correspondiente a solamente el año:

In [25]:
df[1990].columns

Index([ 'Unnamed: 0',          1990,  'Unnamed: 2',  'Unnamed: 3',
        'Unnamed: 4',  'Unnamed: 5',  'Unnamed: 6',  'Unnamed: 7',
        'Unnamed: 8',  'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11',
       'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15',
       'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19',
       'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
       'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27',
       'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31'],
      dtype='object')

Esta fila se descartará ya que es no es indispensable para identificar el año.

La segunda fila lista gases de efecto invernadero y la tercera lista sirve como un desglose de los subtipos de algunos de estos:

In [26]:
l={}
for i in range(0,32):
    if type(df[1990].iloc[0,i]) == str:
        l[i]= df[1990].iloc[0,i]

l

{1: 'CO2',
 2: 'CH4',
 3: 'N2O',
 4: 'HFCs',
 20: 'PFCs',
 26: 'NF3',
 27: 'SF6',
 28: 'EMISIONES NETAS\nGg en CO2e',
 29: ' \nEMISIONES\n(sin 3B y 3D)\nGg en CO2e'}

In [27]:
l={}
for i in range(0,32):
    if type(df[1990].iloc[1,i]) == str:
        l[i]= df[1990].iloc[1,i]

l

{4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32',
 20: 'CF4',
 21: 'C2F6',
 22: 'C3F8',
 23: 'C4F6',
 24: 'c-C4F8',
 25: 'C5F8'}

Los tipos de gases que muestran subtipos son los HFCs y los PFCs ya que son los que se sobreponen con los gases de la fila 2. Para perservar lamayor información posible guardamos en un diccionario una lista de los gases de cada tipo:

In [28]:
HFC={}
for i in range(4,20):
    HFC[i]= df[1990].iloc[1,i]

HFC

{4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32'}

In [29]:
PCF={}
for i in range(20,26):
    PCF[i]= df[1990].iloc[1,i]

PCF

{20: 'CF4', 21: 'C2F6', 22: 'C3F8', 23: 'C4F6', 24: 'c-C4F8', 25: 'C5F8'}

Ahora juntamos ambas filas:

In [30]:
l={}
for i in range(0,32):
    if type(df[1990].iloc[0,i]) == str:
        l[i]= df[1990].iloc[0,i]
    if type(df[1990].iloc[1,i]) == str:
        l[i]= df[1990].iloc[1,i]

l

{1: 'CO2',
 2: 'CH4',
 3: 'N2O',
 4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32',
 20: 'CF4',
 21: 'C2F6',
 22: 'C3F8',
 23: 'C4F6',
 24: 'c-C4F8',
 25: 'C5F8',
 26: 'NF3',
 27: 'SF6',
 28: 'EMISIONES NETAS\nGg en CO2e',
 29: ' \nEMISIONES\n(sin 3B y 3D)\nGg en CO2e'}

Las columnas que quedan sin nombre son las columnas 0, 30 y 31.

La columna 30 aparenta estar vacía por lo que comprobamos. Usamos un ciclo para comprobar todos los datos de la columna:

In [31]:
c=True

for i in (0, len(df[1990])-1):
    if np.isnan(df[1990].iloc[i,30]) == False:
        c=False

c

True

Ya que se comprobó que es una columna vacía es posible desacernos de ella:

In [32]:
for i in range(1990,2022):
    p= df[i].pop(df[i].columns[30])


El ciclo recorre todos los dataframes quitando la columna 30. Se utiliza la función *columns* dentro de la función *pop* para que nos arroje el nombre de la columna ya que la función *pop* requiere el nombre de la columna.

Comprobamos que se haya eliminado la columna:

In [33]:
df[2021]

,Unnamed: 0,2021,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 31
0,NaN,CO2,CH4,N2O,HFCs,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NF3,SF6,EMISIONES NETAS\nGg en CO2e,EMISIONES\n(sin 3B y 3D)\nGg en CO2e,NaN
1,NaN,NaN,NaN,NaN,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NaN,NaN,NaN,NaN,NaN
2,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,11100,8900,1,9540,2,16100,23500,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,EMISIONES NETAS (Gg de CO2e),456265.876441,198041.61451,36315.62913,1974.597954,1670.82773,5.857883,34.841703,0.275913,8421.011818,...,3.233302,0.29995,0.000003,0.107289,0.000002,2.96463,2.498574,714047.264077,714047.264077,66.371687
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182,[4D] Tratamiento y eliminación de aguas residu...,NaN,25804.882349,5571.51224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31376.394589,NaN,NaN
183,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4930.620916,2781.021175,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN
184,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20874.261433,2790.491065,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN
185,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Ahora solo resta completar el nombre de la primera y última columna.

Se decide nombrar a la primera columna como "*Fuente*" y después de una revisión de las bases de datos originales se llega a la conclusión de que la última columna es nombrada "*Carbono negro (Gg)*".

También se decide corregir las columnas 28 y 29.

Para esto usamos nuevamente el diccionario de **l**:

In [34]:
l[0] = 'Fuentes'
l[30] = 'Carbono_negro(Gg)'
l[29] = 'EMISIONES(sin 3B y 3D)(Gg en CO2e)'
l[28] = 'EMISIONES_NETAS(Gg en CO2e)'

l


{1: 'CO2',
 2: 'CH4',
 3: 'N2O',
 4: 'HFC-23',
 5: 'HFC-410A',
 6: 'HFC-43-10mee',
 7: 'HFC-125',
 8: 'HFC-134',
 9: 'HFC-134a',
 10: 'HFC-404A',
 11: 'HFC-407C',
 12: 'HFC-507a',
 13: 'HFC-152a',
 14: 'HFC-227ea',
 15: 'HFC-236fa',
 16: 'HFC-365mfc/227ea',
 17: 'HFC-365mfc',
 18: 'HFC-245fa',
 19: 'HFC-32',
 20: 'CF4',
 21: 'C2F6',
 22: 'C3F8',
 23: 'C4F6',
 24: 'c-C4F8',
 25: 'C5F8',
 26: 'NF3',
 27: 'SF6',
 28: 'EMISIONES_NETAS(Gg en CO2e)',
 29: 'EMISIONES(sin 3B y 3D)(Gg en CO2e)',
 0: 'Fuentes',
 30: 'Carbono_negro(Gg)'}

Ahora podemos ordenar los nombres deseados de las columnas en una lista:

In [35]:
columnas = []
for i in range(0,31):
    columnas.append(l[i])

columnas

['Fuentes',
 'CO2',
 'CH4',
 'N2O',
 'HFC-23',
 'HFC-410A',
 'HFC-43-10mee',
 'HFC-125',
 'HFC-134',
 'HFC-134a',
 'HFC-404A',
 'HFC-407C',
 'HFC-507a',
 'HFC-152a',
 'HFC-227ea',
 'HFC-236fa',
 'HFC-365mfc/227ea',
 'HFC-365mfc',
 'HFC-245fa',
 'HFC-32',
 'CF4',
 'C2F6',
 'C3F8',
 'C4F6',
 'c-C4F8',
 'C5F8',
 'NF3',
 'SF6',
 'EMISIONES_NETAS(Gg en CO2e)',
 'EMISIONES(sin 3B y 3D)(Gg en CO2e)',
 'Carbono_negro(Gg)']

Ahora con la ayuda de esta lista se pueden corregir los nombres de las columnas y eliminar las filas antes señaladas:

In [36]:
df_col_correg = {}
tt = pd.DataFrame

for i in range(1990, 2022):
    tt = df[i].copy()
    tt.columns = columnas
    tt = tt.drop(0)
    tt = tt.drop(1)
    tt = tt.reset_index(drop=True)
    df_col_correg[i] = tt


El código anterior crea un nuevo diccionario para guardar los dataframes ya corregidos llamado **df_col_correg**. La asignación se realiza con la ayuda de una variable auxiliar llamada **tt** en la cuál se asigna una copia de los dataframes correspondientes, se corrigen los nombres de las columnas para después eliminar las filas extras y corregir la indexación.

Imprimimos algunos dataframe para observar que no haya problema:

In [37]:
df_col_correg[1990]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,11100,8900,1,9540,2,16100,23500,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
3,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
4,[1A] Actividades de quema del combustible,277455.844583,2933.343959,2879.060741,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283268.249283,NaN,71.238034
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN
168,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
169,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN
170,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [38]:
df_col_correg[2020]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,Potencial de calentamiento,1,28,265,12400,1924,1650,3170,1120,1300,...,11100,8900,1,9540,2,16100,23500,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,2.696136,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,72.174862
3,[1] Energía,414480.670039,34441.977884,2130.668697,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451053.31662,NaN,64.050795
4,[1A] Actividades de quema del combustible,398693.932566,3495.915494,2119.725053,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404309.573112,NaN,55.570891
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN
168,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN
169,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN
170,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


##### Corrección de filas duplicadas

Al revisar los nuevos dataframes se observa lo siguiente:
* La fila 0 (Potencial de calentamiento) parece ser igual en todos los dataframes ya que corresponde a una característica propia de cada gas
* La fila 1 parece estar vacía
* Las filas 2 y 171 parecen ser iguales

Comprobamos que la fila 0 es igual en todos los dataframes:

In [39]:
c=True
for i in range(1990, 2020):
    for j in range(1,31):
        if df_col_correg[i].iloc[0,j] != df_col_correg[i+1].iloc[0,j]:
            c= False
            break

c

False

Ya que el resultado es 'False' imprimimos *i* y *j* para ver donde se da la diferencia:

In [40]:
print(i, j)

2019 28


Revisamos los datos:

In [41]:
df_col_correg[i+1].iloc[0,j]

nan

Ya que en los datos son del tipo '*nan*' es posible que la naturaleza de los datos ocasionen la diferencia por lo que validamos nuevamente esta vez considerando este tipo de datos:

In [42]:
c=True
for i in range(1990, 2020):
    for j in range(1,31):
        if (np.isnan(df_col_correg[i].iloc[0,j]) and np.isnan(df_col_correg[i+1].iloc[0,j])) == True:
            continue
        if df_col_correg[i].iloc[0,j] != df_col_correg[i+1].iloc[0,j]:
            c= False
            break

c

True

Ya que se validó que la fila es igual en todas de momento se decide quitarla y conservar la información en una variable distinta:

In [43]:
Potencial_de_calentamiento= {}
for i in range(0,31):
    Potencial_de_calentamiento[df_col_correg[1990].columns[i]] = df_col_correg[1990].iloc[0,i]

Potencial_de_calentamiento

{'Fuentes': '  Potencial de calentamiento',
 'CO2': 1,
 'CH4': 28,
 'N2O': 265,
 'HFC-23': 12400,
 'HFC-410A': 1924,
 'HFC-43-10mee': 1650,
 'HFC-125': 3170,
 'HFC-134': 1120,
 'HFC-134a': 1300,
 'HFC-404A': 3943,
 'HFC-407C': 1624,
 'HFC-507a': 3985,
 'HFC-152a': 138,
 'HFC-227ea': 2640,
 'HFC-236fa': 8060,
 'HFC-365mfc/227ea': 982,
 'HFC-365mfc': 804,
 'HFC-245fa': 858,
 'HFC-32': 677,
 'CF4': 6630,
 'C2F6': 11100,
 'C3F8': 8900,
 'C4F6': 1,
 'c-C4F8': 9540,
 'C5F8': 2,
 'NF3': 16100,
 'SF6': 23500,
 'EMISIONES_NETAS(Gg en CO2e)': nan,
 'EMISIONES(sin 3B y 3D)(Gg en CO2e)': nan,
 'Carbono_negro(Gg)': nan}

Una vez que la información se guardó en un diccionario separado, removemos la fila de todos los dataframes:

In [44]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].drop(0)

In [45]:
df_col_correg[2019]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,2.629593,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,65.581718
3,[1] Energía,444504.90145,21170.54218,2216.930109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,467892.373739,NaN,54.963710
4,[1A] Actividades de quema del combustible,432110.270818,2821.764602,2182.895974,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437114.931394,NaN,48.860462
5,[1A1] Industrias de la energía,202816.918337,173.965545,303.316786,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203294.200668,NaN,8.039951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN
168,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN
169,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN
170,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Reseteamos los indices para que no haya confusiones:

In [46]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].reset_index(drop=True)

In [47]:
df_col_correg[2019]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EMISIONES NETAS (Gg de CO2e),295777.936009,175558.468175,41190.822329,1388.105399,1848.068899,3.102067,37.805667,0.280098,7778.004868,...,2.629593,0.245714,0.000003,0.08791,0.000002,2.474452,400.915891,534688.600102,736629.572741,65.581718
2,[1] Energía,444504.90145,21170.54218,2216.930109,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,467892.373739,NaN,54.963710
3,[1A] Actividades de quema del combustible,432110.270818,2821.764602,2182.895974,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,437114.931394,NaN,48.860462
4,[1A1] Industrias de la energía,202816.918337,173.965545,303.316786,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,203294.200668,NaN,8.039951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,[4D] Tratamiento y eliminación de aguas residu...,NaN,20597.214693,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23109.465677,NaN,NaN
167,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4191.354013,2512.250984,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN
168,[4D2] Tratamiento y eliminación de aguas resid...,NaN,16405.860681,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN
169,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Ahora procedemos a verificar que la fila 1 este vacía:

In [48]:
c= False
for i in range(0,31):
    if np.isnan(df_col_correg[1990].iloc[0,i]) == False:
        c = True

c

False

Ya que comprobamos que está vacía podemos eliminarla de los dataframes:

In [49]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].drop(0)
    df_col_correg[i]=df_col_correg[i].reset_index(drop=True)
    

In [50]:
df_col_correg[2020]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),464170.571806,193571.274851,37336.746306,513.405854,2114.932735,7.004317,36.293441,0.277997,8127.355593,...,2.696136,0.244027,0.000003,0.087292,0.000002,2.424645,2.092453,716684.648009,716684.648009,72.174862
1,[1] Energía,414480.670039,34441.977884,2130.668697,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,451053.31662,NaN,64.050795
2,[1A] Actividades de quema del combustible,398693.932566,3495.915494,2119.725053,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404309.573112,NaN,55.570891
3,[1A1] Industrias de la energía,174609.239141,139.906396,213.414841,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,174962.560377,NaN,5.401026
4,[1A1a] Actividad principal producción de elect...,147501.152024,125.631395,197.19366,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147823.977079,NaN,4.828733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,[4D] Tratamiento y eliminación de aguas residu...,NaN,24851.535461,6677.909224,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31529.444684,NaN,NaN
166,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4768.077185,4157.694446,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8925.771631,NaN,NaN
167,[4D2] Tratamiento y eliminación de aguas resid...,NaN,20083.458276,2520.214777,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22603.673053,NaN,NaN
168,[4E] Otros,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


Ahora comprobamos si la fila 0 y 169 son iguales para todos los dataframes:

In [51]:
c=True
for i in range(1990,2022):
    for j in range(1,31):
        if (np.isnan(df_col_correg[i].iloc[0,j]) and np.isnan(df_col_correg[i].iloc[169,j])) == True:
            continue
        if df_col_correg[i].iloc[0,j] != df_col_correg[i].iloc[169,j]:
            c=False
            break

c

False

Vemos que no son iguales así que comprobamos los vañores de *i* y *j* así como los valores que se marcan como diferentes

In [52]:
print(i,j)

2021 1


In [53]:
print(df_col_correg[i].iloc[0,j],df_col_correg[i].iloc[169,j])

456265.8764410899 456265.87644108996


Vemos que las diferencias son bastante pequeñas. Comprobamos que este sea el caso para todos los valores y dataframes:

In [54]:
dif= []

for i in range(1990,2022):
    aux=[]
    for j in range(1,31):
        aux.append(df_col_correg[1990].iloc[0,j] - df_col_correg[1990].iloc[169,j])
    
    dif.append(np.nanmean(aux))


El ciclo calcula las diferencias entre cada fila y lo guarda en una lista auxiliar para después obtener el promedio de las diferencias de cada dataframe y guardarla en la lista **dif**. Ahora podemos ver cuál es el valor máximo de esa lista y verificar si es una diferencia significativa:

In [55]:
np.max(dif)

4.014811471781035e-12

Ya que el valor es muy pequeño no se consideran diferencias significativas por lo que se elimina una de ellas.

In [56]:
for i in range(1990,2022):
    df_col_correg[i]=df_col_correg[i].drop(169)
    df_col_correg[i]=df_col_correg[i].reset_index(drop=True)

In [57]:
df_col_correg[1990]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
2,[1A] Actividades de quema del combustible,277455.844583,2933.343959,2879.060741,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283268.249283,NaN,71.238034
3,[1A1] Industrias de la energía,107765.12174,92.051436,187.539901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108044.713077,NaN,14.152478
4,[1A1a] Actividad principal producción de elect...,70302.750576,62.745182,139.850236,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70505.345994,NaN,11.656345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,[4C2] Quema a cielo abierto de residuos sólidos,402.492892,801.997327,175.161504,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379.651724,NaN,1.354876
165,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN
166,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
167,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN


##### Corrección de formato de valores

Para evitar futuros errores se eliminan los espacios al inicio y final de las cadenas de texto de la columna 0:

In [58]:
for i in range(1990,2022):
    for j in range(0,169):
        df_col_correg[i].iloc[j,0] = df_col_correg[i].iloc[j,0].strip()

df_col_correg[2019].iloc[:,0]

0                           EMISIONES NETAS (Gg de CO2e)
1                                            [1] Energía
2              [1A] Actividades de quema del combustible
3                         [1A1] Industrias de la energía
4      [1A1a] Actividad principal producción de elect...
                             ...                        
164      [4C2] Quema a cielo abierto de residuos sólidos
165    [4D] Tratamiento y eliminación de aguas residu...
166    [4D1] Tratamiento y eliminación de aguas resid...
167    [4D2] Tratamiento y eliminación de aguas resid...
168                                           [4E] Otros
Name: Fuentes, Length: 169, dtype: object

También se revisa el tipo de dato para el resto de las variables. 

Se sabe que las columnas de valores beben de ser por completo del tipo flotante.

In [59]:
c= True

for i in range(1990,2022):
    for j in range(0,169):
        if type(df_col_correg[i].iloc[j,1]) != float:
            c = False

c

False

Como vemos se tiene que no todo los datos de al menos una columna es flotante. Probablemente sean variables de tipo *string* y sean espacios vacions o *'NAs'*:

In [60]:
c= False
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if type(df_col_correg[i].iloc[h,j]) == str:
                c= True

c

True

Probamos imprimir estos datos a *float*:

In [61]:
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if type(df_col_correg[i].iloc[h,j]) == str:
                print(df_col_correg[i].iloc[h,j])

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE
NE


En base a la imprensión podemos ver que en su mayoría son eldas con espacios y *'NE'*.

Cambiamos los espacios vacios y *'NE'* a *'nan'*:

In [62]:
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if (df_col_correg[i].iloc[h,j] == 'NE' or df_col_correg[i].iloc[h,j] == ' '):
                df_col_correg[i].iloc[h,j] = 'nan'

Comprobamos los cambios:

In [63]:
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if type(df_col_correg[i].iloc[h,j]) == str:
                print(df_col_correg[i].iloc[h,j])

nan


nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


Cambiamos los datos de tipo:

In [64]:
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if type(df_col_correg[i].iloc[h,j]) == str:
                df_col_correg[i].iloc[h,j]=float(df_col_correg[i].iloc[h,j])

Ahora revisamos el tipo una vez más:

In [65]:
c= False
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if type(df_col_correg[i].iloc[h,j]) != float:
                c= True

c

True

Se revisa el tipo diferente a *float*:

In [66]:
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if type(df_col_correg[i].iloc[h,j]) != float:
                print(df_col_correg[i].iloc[h,j])
                print(type(df_col_correg[i].iloc[h,j]))

0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class 'int'>
0
<class

Los datos tipo *int* no representan problemas con el tipo *float* por lo que solo se cofirma que existan solo estos tipos:

In [67]:
c= False
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if (type(df_col_correg[i].iloc[h,j]) != float and type(df_col_correg[i].iloc[h,j]) != int):
                c= True

c

True

In [68]:
c= False
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if (type(df_col_correg[i].iloc[h,j]) != float and type(df_col_correg[i].iloc[h,j]) != int):
                print(type(df_col_correg[i].iloc[h,j]))

<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.float64'>
<class 'numpy.fl

Variables del tipo *np.float64* tampoco darán problemas así que validamos nuevamente:

In [69]:
c= False
for i in range(1990,2022):
    for j in range(1,31):
        for h in range(0,169):
            if (type(df_col_correg[i].iloc[h,j]) != float and type(df_col_correg[i].iloc[h,j]) != int and type(df_col_correg[i].iloc[h,j]) != np.float64):
                c= True

c

False

### Reorganización de datos

#### Clasificación de filas por niveles

Ya que algunas filas son correspondientes a una subclase de otras se considera conveniente saber en que nivel jerarquico se encuentra cada una de ellas. 

Para esto se usará el hecho de que en la mayoría de los casos la subdivisión está dada entre corchetes (**[ ]**).

La base tiene 4 grandes categorías, identificadas con los números del 1 al 4, a las cuales se les agregan letras y números para identificar las subcategorías correspondientes a cada una. Es decir la categoría *[1]* se divide a su vez en *[1A]* y *[1B]*. A su vez *[1A]* se divide en *[1A1]*, *[1A2]*, *[1A3]* y *[1A4]*. Después *[1A1]* se divide en *[1A1a]*, *[1A1b]* y *[1A1c]*. Finalmente al llegar a un quinto nivel el desglose se quita de los corchetes, por ejemplo, la categoría *[1A1c]* se divide en *1A1ci* y *1A1cii*.

Podemos usar la cantidad de caracteres dentro de los corchetes para encontrar el nivel de clasificación as filas:

In [70]:
clasif = {}

for i in range(0,169):
 #clasif.append(df_col_correg[1990].iloc[i,0].find(']')-df_col_correg[1990].iloc[i,0].find('['))
 clasif[df_col_correg[1990].iloc[i,0]] = df_col_correg[1990].iloc[i,0].find(']')-df_col_correg[1990].iloc[i,0].find('[')-1

clasif

{'EMISIONES NETAS (Gg de CO2e)': -1,
 '[1] Energía': 1,
 '[1A] Actividades de quema del combustible': 2,
 '[1A1] Industrias de la energía': 3,
 '[1A1a] Actividad principal producción de electricidad y calor': 4,
 '[1A1b] Refinación del petróleo': 4,
 '[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía': 4,
 '[1A2] Industrias manufactura y de la construcción': 3,
 '[1A2a] Hierro y acero': 4,
 '[1A2b] Metales no ferrosos': 4,
 '[1A2c] Sustancias químicas': 4,
 '[1A2d] Pulpa, papel e imprenta': 4,
 '[1A2e] Procesamiento de alimentos, bebidas y tabaco': 4,
 '[1A2f] Minerales no metálicos': 4,
 '[1A2g] Equipo de transporte': 4,
 '[1A2h] Maquinaria': 4,
 '[1A2i] Minería (con excepción de combustibles) y cantería': 4,
 '[1A2j] Madera y productos de la madera': 4,
 '[1A2k] Construcción': 4,
 '[1A2l] Textiles y cueros': 4,
 '[1A2m] Industria no especificada': 4,
 '[1A3] Transporte': 3,
 '[1A3a] Aviación civil': 4,
 '[1A3b] Autotransporte': 4,
 '[1A3c] Ferrocarriles': 4,

Del diccionario anterior vemos que los resutados están relacionados con el nivel de clasificación:
- *[1]* primer nivel: 1
- *[1A]* segundo nivel: 2
- *[1A1]* tercer nivel: 3
- *[1A1a]* cuarto nivel: 6

Como se ve a continuación, se tienen también resultados de 5 y 6 pero estos pertenecen a un quinto nivel:
- [1B1ai]
- [1B1aii]

In [71]:
for i in clasif.keys():
    if clasif[i] > 4:
     print (clasif[i], i)

5 [1B1ai] Minas subterráneas
6 [1B1aii] Minas superficie


Los valores de -1 se imprimen para saber si pertenecen a algun nivel de clasificación:

In [72]:
for i in clasif.keys():
    if clasif[i] == -1:
     print (clasif[i], i)

-1 EMISIONES NETAS (Gg de CO2e)
-1 1B2ai Venteo petróleo
-1 1B2aii Quemado petróleo
-1 1B2aiii Otras fugitivas petróleo
-1 1B2bi Venteo gas natural
-1 1B2bii Quemado gas natural
-1 1B2biii Otras fugitivas gas natural


Como podemos ver todos excepcto el correspondiente a la *fila 0* corresponden a clasificación de quinto nivel.

Arreglamos la clasificación de estos casos:

In [73]:
#se cambian los todos los 0 por 5
for i in clasif.keys():
    if clasif[i] == -1:
     clasif[i] = 5
     print (clasif[i], i)

5 EMISIONES NETAS (Gg de CO2e)
5 1B2ai Venteo petróleo
5 1B2aii Quemado petróleo
5 1B2aiii Otras fugitivas petróleo
5 1B2bi Venteo gas natural
5 1B2bii Quemado gas natural
5 1B2biii Otras fugitivas gas natural


In [74]:
#se cambia el caso especifico que debe permanecer como 0
clasif["EMISIONES NETAS (Gg de CO2e)"] = 0

In [75]:
#Nos aseguramos que solo Emisiones quede en 0 y ya no haya -1

for i in clasif.keys():
    if clasif[i] == 0:
     print (clasif[i], i)
    elif clasif[i] == -1:
     print (clasif[i], i)

0 EMISIONES NETAS (Gg de CO2e)


In [76]:
# cambiamos el caso particular de  6
for i in clasif.keys():
    if clasif[i] == 6:
     clasif[i] = 5
     print (clasif[i], i)

5 [1B1aii] Minas superficie


In [77]:
#Nos aseguramos que 5 sea el nivel máximo y el mínimo sea 0
print(max(clasif.values()),min(clasif.values()))

5 0


En posteriores revisiones se descubrió que hay un caso especial en los niveles 4. Existe una categoría *[2B10]* que por el número de caracteres se clasifica como 4 pero pertenece al nivel 3:

In [78]:
clasif['[2B10] Otros']

4

Para corregirlo se hace una asignación individual:

In [79]:
clasif['[2B10] Otros'] = 3

#### Separación de niveles en columnas

Ahora que tenemos una clasificación correcta podemos separar cada una de las categorías y subcategorías en diferentes columnas.

Esto para tener datos en su forma más básica sin que ninguna fila sea el subtotal de otras.

Nótese que previamente se había confirmado que todos los dataframes por año comparten las mismas filas, por lo que es seguro decir que el resultado generado a continuación será el mismo para todos los años. En base a esto, no es importante bajo que año se realicen los procedimientos por lo que se elige usar el año de 1990 aleatoriamente.

Usaremos un diccionario para guardar los niveles de cada fila en 5 columnas:

- categoría : nivel 1   *[1]*
- fuente : nivel 2  *[1A]*
- subfuente 1: nivel 3  *[1A1]*
- subfuente 2: nivel 4  *[1A1a]*
- subfuente 3: nivel 5  *[1A1ai]*

In [80]:
categs = {}

for fila in range(1,168):
    clasifactual = clasif[df_col_correg[1990].iloc[fila,0]]
    if clasifactual == 1:
        categoria = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 2:
        fuente = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 3:
        subn1 = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 4:
        subn2 = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 5:
        subn3 = df_col_correg[1990].iloc[fila,0]
    
    if clasifactual > clasif[df_col_correg[1990].iloc[fila+1,0]]:
        if clasifactual == 2:
            categs[fila] = [categoria,fuente,"-","-","-"]
        if clasifactual == 3:
            categs[fila] = [categoria,fuente,subn1,"-","-"]
        if clasifactual == 4:
            categs[fila] = [categoria,fuente,subn1,subn2,"-"]
        if clasifactual == 5:
            categs[fila] = [categoria,fuente,subn1,subn2,subn3]
        
        cont = fila +1-1
        while clasif[df_col_correg[1990].iloc[cont,0]] == clasif[df_col_correg[1990].iloc[cont-1,0]]:
            if clasif[df_col_correg[1990].iloc[cont-1,0]] == 2:
               fuente = df_col_correg[1990].iloc[cont-1,0]
               categs[cont-1] = [categoria,fuente,"-","-","-"]
            elif clasif[df_col_correg[1990].iloc[cont-1,0]] == 3:
                subn1 = df_col_correg[1990].iloc[cont-1,0]
                categs[cont-1] = [categoria,fuente,subn1,"-","-"]
            elif clasif[df_col_correg[1990].iloc[cont-1,0]] == 4:
                subn2 = df_col_correg[1990].iloc[cont-1,0]
                categs[cont-1] = [categoria,fuente,subn1,subn2,"-"]
            elif clasif[df_col_correg[1990].iloc[cont-1,0]] == 5:
                subn3 = df_col_correg[1990].iloc[cont-1,0]
                categs[cont-1] = [categoria,fuente,subn1,subn2,subn3]
            
            cont = cont-1

Para entender el código anterior es necesario tener en cuenta que las filas de subtotales siempre vienen antes de su desgloce. Es decir el total de la categoría se presenta primero y en la siguiente fila el total de la primera fuente. En las filas siguientes del total de la primera fuente se pueden presentar el total de la primera subfuente y después el desgloce de esta subfuente en otras y así sucesivamente.

Bajo esta organización en el diccionario de clasificación se ve un aumento de niveles constante, por ejemplo [1,2,3,4,4,4,5,5,3], y cada vez que se reduzca el nivel es debido a que a que ya se ha realizado por completo el desgloce de una fuente. 

El código recorre todas las filas del dataframe (excepto la fila 0 y la última fila) y dependiendo del nivel de cada fila asigna su valor a una de las cinco variables *categoria*, *fuente*, *subn1*, *subn2* o *subn3* de esta manera se van guardando cada subcategoría conforme se recorren.

En el caso de que el nivel de la siguinete fila sea menor al presente (indicando el desgloce total de una fuente) revisa el nivel presente para asignar la lista correspondiente:
- si es nivel 2 se agrega una lista con *categoria* y *fuente* y se llenan las subfuentes con "-".
- si es nivel 3 se agrega una lista con *categoria*, *fuente* y *subn1* y el resto de las subfuentes se reellenan con "-".
- si es nivel 4 se agrega una lista con *categoria*, *fuente*, *subn1* y *subn2* y la subfuente restante se reellena con "-".
- si es nivel 5 se agrega una lista con todas las variables *categoria*, *fuente*, *subn1*, *subn2* y *subn3*.

Después de eso se hace la pregunta por medio de un *while* si el nivel de la fila anterior es igual al presente para casos en que se hayan dado filas del mismo nivel consecutivas. Si la respuesta es que si entonces obtiene el valor y hace la asignación por categoría segun el nivel.

Uno de los problemas que se pueden dar es en el caso de que solo el último de una secuencia de mismos niveles tenga un desgloce adicional.
Por ejemplo, si se tiene [1,2,3,4,4,4,5,5,3] al llegar al 5 el código solo reconocerá y asignará los del mismo nivel pero ignorará los 4 sucesivos.

Para corregir esto se usará el mismo *while* una vez más. Se hace la pregunta si *cont-1* y *cont-2* son iguales para iniciar el ciclo nuevamente.

Para facilitar esto podemos poner el ciclo en una función.

In [81]:
# Función para asignar las categorías de niveles anteriores

def whil_ant(num, categoria, fuente, subn1, subn2, subn3):
    while clasif[df_col_correg[1990].iloc[num,0]] == clasif[df_col_correg[1990].iloc[num-1,0]]:
        if clasif[df_col_correg[1990].iloc[num-1,0]] == 2:
               fuente = df_col_correg[1990].iloc[num-1,0]
               categs[num-1] = [categoria,fuente,"-","-","-"]
        elif clasif[df_col_correg[1990].iloc[num-1,0]] == 3:
                subn1 = df_col_correg[1990].iloc[num-1,0]
                categs[num-1] = [categoria,fuente,subn1,"-","-"]
        elif clasif[df_col_correg[1990].iloc[num-1,0]] == 4:
                subn2 = df_col_correg[1990].iloc[num-1,0]
                categs[num-1] = [categoria,fuente,subn1,subn2,"-"]
        elif clasif[df_col_correg[1990].iloc[num-1,0]] == 5:
                subn3 = df_col_correg[1990].iloc[num-1,0]
                categs[num-1] = [categoria,fuente,subn1,subn2,subn3]

        num = num-1   

    return num

La función trabaja de la misma manera que el *while* dentro del primer código y regresa el contador actual. -->

Ahora que se tiene la función se puede agregar al código:

In [82]:
categs = {}

for fila in range(1,168):
    clasifactual = clasif[df_col_correg[1990].iloc[fila,0]]
    if clasifactual == 1:
        categoria = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 2:
        fuente = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 3:
        subn1 = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 4:
        subn2 = df_col_correg[1990].iloc[fila,0]
    elif clasifactual == 5:
        subn3 = df_col_correg[1990].iloc[fila,0]
    
    if clasifactual > clasif[df_col_correg[1990].iloc[fila+1,0]]:
        if clasifactual == 2:
            categs[fila] = [categoria,fuente,"-","-","-"]
        if clasifactual == 3:
            categs[fila] = [categoria,fuente,subn1,"-","-"]
        if clasifactual == 4:
            categs[fila] = [categoria,fuente,subn1,subn2,"-"]
        if clasifactual == 5:
            categs[fila] = [categoria,fuente,subn1,subn2,subn3]
        
        cont = fila +1-1
        cont = whil_ant(cont,categoria, fuente, subn1, subn2, subn3)

        if clasif[df_col_correg[1990].iloc[cont-1,0]] == clasif[df_col_correg[1990].iloc[cont-2,0]]:
            cont = whil_ant(cont-1,categoria, fuente, subn1, subn2, subn3)
        
        if clasif[df_col_correg[1990].iloc[cont-1,0]] == clasif[df_col_correg[1990].iloc[cont-2,0]]:
            cont = whil_ant(cont-1,categoria, fuente, subn1, subn2, subn3)
        
        if clasif[df_col_correg[1990].iloc[cont-1,0]] == clasif[df_col_correg[1990].iloc[cont-2,0]]:
            cont = whil_ant(cont-1,categoria, fuente, subn1, subn2, subn3)

La asignación se realiza dentro de la función y se devuelve el contador para mantener el actualizado el dato dentro del código original. 

Después de la asignación de los niveles actual se hace la pregunta si el nivel del *con-1* y el *cont-2* son iguales y de ser así se hace una vez más el ciclo.

Se hace esta pregunta 3 veces para abarcar la posibilidad de que se tengan secuencias desde el número 2 hasta el 5: [1,2,2,3,3,3,4,4,4,5,5,3] por ejemplo.

Una vez terminado, podemos imprimir el diccionario para ver las asignaciones:

In [83]:
categs

{6: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A1] Industrias de la energía',
  '[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía',
  '-'],
 5: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A1] Industrias de la energía',
  '[1A1b] Refinación del petróleo',
  '-'],
 4: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A1] Industrias de la energía',
  '[1A1a] Actividad principal producción de electricidad y calor',
  '-'],
 20: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A2] Industrias manufactura y de la construcción',
  '[1A2m] Industria no especificada',
  '-'],
 19: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A2] Industrias manufactura y de la construcción',
  '[1A2l] Textiles y cueros',
  '-'],
 18: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A2] Industrias manufactura y de la construcción',
  '[1A2k] Construcción',
  '-'

Como podemos observar tal y como se había comentado anteriormente, el código no abarca el primer y último elemento. Debido a esto se hacen las asignaciónes individuales:

In [84]:
#agregamos la primera y ultima fila 
categs[0] = [df_col_correg[1990].iloc[0,0],"-","-","-","-"]
categs[168] = [categoria,df_col_correg[1990].iloc[168,0],"-","-","-"]

Por como funciona el código se tienen las asignaciones por fila de manera desordenada. Se usa la función *sorted* y un *for* para ordenarlo en un nuevo diccionario.

In [85]:
#acomodamos las llaves del diccionario (filas)
keys_orden = sorted(categs.keys())
keys_orden

[0,
 4,
 5,
 6,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 22,
 23,
 24,
 25,
 26,
 28,
 29,
 30,
 34,
 35,
 36,
 39,
 40,
 41,
 43,
 44,
 45,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 78,
 79,
 80,
 81,
 82,
 84,
 85,
 86,
 87,
 88,
 89,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 124,
 125,
 127,
 128,
 130,
 131,
 133,
 134,
 136,
 137,
 139,
 140,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 154,
 155,
 158,
 159,
 160,
 161,
 163,
 164,
 166,
 167,
 168]

In [86]:
#nuevo diccionario con keys ordenadas

categs_orden = {}
for i in keys_orden:
   # aux = [i]
    #aux.extend(categs[i])
    categs_orden[i] = categs[i]

categs_orden

{0: ['EMISIONES NETAS (Gg de CO2e)', '-', '-', '-', '-'],
 4: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A1] Industrias de la energía',
  '[1A1a] Actividad principal producción de electricidad y calor',
  '-'],
 5: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A1] Industrias de la energía',
  '[1A1b] Refinación del petróleo',
  '-'],
 6: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A1] Industrias de la energía',
  '[1A1c] Manufactura de combustibles sólidos y otras industrias de la energía',
  '-'],
 8: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A2] Industrias manufactura y de la construcción',
  '[1A2a] Hierro y acero',
  '-'],
 9: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A2] Industrias manufactura y de la construcción',
  '[1A2b] Metales no ferrosos',
  '-'],
 10: ['[1] Energía',
  '[1A] Actividades de quema del combustible',
  '[1A2] Industrias manufactura y de l

Una vez ordenado se crea un dataframe para facilitar su visualización y uso:

In [87]:
categss = pd.DataFrame.from_dict(categs_orden, orient = 'index')
categss = categss.reset_index()
categss

,index,0,1,2,3,4
0,0,EMISIONES NETAS (Gg de CO2e),-,-,-,-
1,4,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-
2,5,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-
3,6,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-
4,8,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-
...,...,...,...,...,...,...
125,163,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-
126,164,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-
127,166,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-
128,167,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-


Cambiamos los nombres de las columnas:

In [88]:
categss.columns = ["Fila", "Categoria","Fuente","Subfuente1","Subfuente2","Subfuente3"]
categss

,Fila,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3
0,0,EMISIONES NETAS (Gg de CO2e),-,-,-,-
1,4,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-
2,5,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-
3,6,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-
4,8,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-
...,...,...,...,...,...,...
125,163,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-
126,164,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-
127,166,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-
128,167,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-


Imprimimos las primeras 30 filas para verificar las asignaciones:

In [89]:
categss.iloc[0:30,:]

,Fila,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3
0,0,EMISIONES NETAS (Gg de CO2e),-,-,-,-
1,4,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-
2,5,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-
3,6,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-
4,8,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-
5,9,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2b] Metales no ferrosos,-
6,10,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2c] Sustancias químicas,-
7,11,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,"[1A2d] Pulpa, papel e imprenta",-
8,12,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,"[1A2e] Procesamiento de alimentos, bebidas y t...",-
9,13,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2f] Minerales no metálicos,-


#### Combinación de datos con dataframe de clasificaciones

##### Prueba

Ahora que tenemos el dataframe para las clasificaciones podemos hacer una combinación de los dataframes por años y la columna de clasificaciones usando la función *merge*.

Comenzamos con una prueba para el año de 1990:

In [90]:
#creamos un dataframe de prueba

df_1990_p = df_col_correg[1990].copy()
df_1990_p

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
2,[1A] Actividades de quema del combustible,277455.844583,2933.343959,2879.060741,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,283268.249283,NaN,71.238034
3,[1A1] Industrias de la energía,107765.12174,92.051436,187.539901,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108044.713077,NaN,14.152478
4,[1A1a] Actividad principal producción de elect...,70302.750576,62.745182,139.850236,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70505.345994,NaN,11.656345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,[4C2] Quema a cielo abierto de residuos sólidos,402.492892,801.997327,175.161504,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379.651724,NaN,1.354876
165,[4D] Tratamiento y eliminación de aguas residu...,NaN,9877.340649,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13367.176323,NaN,NaN
166,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
167,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN


In [91]:
# realizamos la combinación con la función merge()
temp = categss.copy()
df_categ_col = temp.merge(df_1990_p,how="left",left_on="Fila",right_index=True)
df_categ_col

,Fila,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3,Fuentes,CO2,CH4,N2O,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,0,EMISIONES NETAS (Gg de CO2e),-,-,-,-,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,4,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-,[1A1a] Actividad principal producción de elect...,70302.750576,62.745182,139.850236,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70505.345994,NaN,11.656345
2,5,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-,[1A1b] Refinación del petróleo,11550.924752,11.131438,20.203821,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11582.260011,NaN,2.190852
3,6,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-,[1A1c] Manufactura de combustibles sólidos y o...,25911.446413,18.174816,27.485844,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25957.107072,NaN,0.305281
4,8,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-,[1A2a] Hierro y acero,5050.957286,3.683582,5.64817,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5060.289038,NaN,0.122834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,163,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-,[4C1] Incineración de residuos peligrosos indu...,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
126,164,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-,[4C2] Quema a cielo abierto de residuos sólidos,402.492892,801.997327,175.161504,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379.651724,NaN,1.354876
127,166,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-,[4D1] Tratamiento y eliminación de aguas resid...,NaN,4552.253721,3489.835674,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
128,167,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-,[4D2] Tratamiento y eliminación de aguas resid...,NaN,5325.086928,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN


La combinación anterior se hace bajo usando el dataframe de clasificación como *izquierdo* y el dataframe del año de 1990 como *derecho*. Se usa una unión *izquierda* es decir que se conservan los elementos del elemento *izquierdo* añadiendo los del elemento *derecho* que correspondan. La combinación usa la columna **Fila** del elemento *izquierdo* y el index del elemento *derecho*.

La presencia de la columna **Fuentes** nos ayuda a compararla con las nuevas columnas para ver que se ha aplicado el merge correctamente y una vez comprobado se eliminan las columnas **Fuentes** y **Filas**:

In [92]:
df_categ_col = df_categ_col.drop(columns="Fila")
df_categ_col = df_categ_col.drop(columns="Fuentes")

Imprimimos el dataframe resultante para verificar los resultados:

In [93]:
df_categ_col

,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3,CO2,CH4,N2O,HFC-23,HFC-410A,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),-,-,-,-,319999.845468,117840.420104,28337.973,760.63584,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-,70302.750576,62.745182,139.850236,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70505.345994,NaN,11.656345
2,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-,11550.924752,11.131438,20.203821,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11582.260011,NaN,2.190852
3,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-,25911.446413,18.174816,27.485844,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25957.107072,NaN,0.305281
4,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-,5050.957286,3.683582,5.64817,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5060.289038,NaN,0.122834
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-,0,0,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
126,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-,402.492892,801.997327,175.161504,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1379.651724,NaN,1.354876
127,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-,NaN,4552.253721,3489.835674,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8042.089395,NaN,NaN
128,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-,NaN,5325.086928,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5325.086928,NaN,NaN


##### Aplicación a todos los años

Ahora que se ha comprobado que funciona se puede aplicar el mismo procedimiento para todos los años:

In [94]:
df_categ_col = {}

for i in range(1990,2022):
    temp= categss.copy()
    aux = pd.DataFrame
    aux = temp.merge(df_col_correg[i],how="left",left_on="Fila",right_index=True)
    aux = aux.drop(columns="Fila")
    aux = aux.drop(columns="Fuentes")

    df_categ_col[i] = aux.copy()

In [95]:
df_categ_col[2018]

,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3,CO2,CH4,N2O,HFC-23,HFC-410A,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),-,-,-,-,332067.785561,173705.911917,42778.196744,1880.023917,1486.685979,...,2.508326,0.225081,0.000003,0.080531,0.000002,2.272453,278.664096,569873.71023,765600.794791,70.732504
1,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-,167973.687355,138.100722,280.456832,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,168392.244909,NaN,7.121635
2,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-,9217.34561,6.180298,8.841196,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9232.367105,NaN,0.742019
3,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-,21603.179694,10.379912,10.699201,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21624.258807,NaN,0.126341
4,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-,5268.598425,2.613912,2.57421,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5273.786547,NaN,0.016975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-,43.867508,0.076232,1.364662,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.308402,NaN,NaN
126,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-,1016.800276,772.676718,182.677733,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972.154727,NaN,1.590980
127,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-,NaN,4058.609161,2188.201352,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6246.810513,NaN,NaN
128,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-,NaN,16857.351109,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16857.351109,NaN,NaN


#### Creación de una sola tabla para todos los datos

Si se agregan a los dataframes una columna para año y se elimina la fila 0 (total de emisiones por año), es posible unir todos los dataframes en uno solo.

Primero agregamos la columna de año a los dataframes:

In [96]:
df_categ_col_año = {}

for i in range(1990,2022):
    aux = pd.DataFrame
    aux = df_categ_col[i].copy()
    aux = aux.drop(index= 0)
    aux = aux.reset_index(drop=True)
    aux.insert(loc=0,column="Año",value=i)
    df_categ_col_año[i] = aux.copy()


In [97]:
df_categ_col_año[2019]

,Año,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3,CO2,CH4,N2O,HFC-23,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,2019,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-,170956.119945,157.235787,284.030707,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,171397.386439,NaN,7.280405
1,2019,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-,10238.677968,6.498651,8.833485,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10254.010104,NaN,0.683294
2,2019,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-,21622.120424,10.231107,10.452594,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21642.804125,NaN,0.076252
3,2019,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-,5689.028972,2.827496,2.793842,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5694.65031,NaN,0.019361
4,2019,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2b] Metales no ferrosos,-,1854.60332,1.293936,1.919925,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1857.817181,NaN,0.013358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,2019,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-,54.963144,0.093587,2.105061,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57.161791,NaN,NaN
125,2019,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-,982.945452,746.950098,176.595396,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1906.490946,NaN,1.538008
126,2019,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-,NaN,4191.354013,2512.250984,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6703.604997,NaN,NaN
127,2019,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-,NaN,16405.860681,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16405.860681,NaN,NaN


Ahora los concatenamos:

In [98]:
df_todos = df_categ_col_año[1990].copy()

for i in range(1991,2022):
    df_todos = pd.concat([df_todos,df_categ_col_año[i]],axis=0,join='inner', ignore_index=True)

In [99]:
df_todos

,Año,Categoria,Fuente,Subfuente1,Subfuente2,Subfuente3,CO2,CH4,N2O,HFC-23,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,1990,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1a] Actividad principal producción de elect...,-,70302.750576,62.745182,139.850236,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70505.345994,NaN,11.656345
1,1990,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1b] Refinación del petróleo,-,11550.924752,11.131438,20.203821,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11582.260011,NaN,2.190852
2,1990,[1] Energía,[1A] Actividades de quema del combustible,[1A1] Industrias de la energía,[1A1c] Manufactura de combustibles sólidos y o...,-,25911.446413,18.174816,27.485844,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25957.107072,NaN,0.305281
3,1990,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2a] Hierro y acero,-,5050.957286,3.683582,5.64817,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5060.289038,NaN,0.122834
4,1990,[1] Energía,[1A] Actividades de quema del combustible,[1A2] Industrias manufactura y de la construcción,[1A2b] Metales no ferrosos,-,2105.775916,1.87628,2.920578,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2110.572775,NaN,0.038763
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4123,2021,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C1] Incineración de residuos peligrosos indu...,-,-,28.597444,0.041421,0.762761,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.401625,NaN,NaN
4124,2021,[4] Residuos,[4C] Incineración y quema a cielo abierto de ...,[4C2] Quema a cielo abierto de residuos sólidos,-,-,1705.641772,754.454674,164.777875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2624.874321,NaN,1.435
4125,2021,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D1] Tratamiento y eliminación de aguas resid...,-,-,NaN,4930.620916,2781.021175,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7711.642091,NaN,NaN
4126,2021,[4] Residuos,[4D] Tratamiento y eliminación de aguas residu...,[4D2] Tratamiento y eliminación de aguas resid...,-,-,NaN,20874.261433,2790.491065,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23664.752498,NaN,NaN


Podemos guardar esto en un archivo de Excel:

In [101]:
df_todos.to_excel("C:/Users/patty/Documents/Maestria Base de Datos/Aprendizaje_Automatico_priv/todos_datos.xlsx",sheet_name='TODOS',index=False)

### PRUEBAS FALLIDAS

####### Función para asignar las categorías de niveles anteriores por año
#

def whil_ant(yy,num, categoria, fuente, subn1, subn2, subn3):
    while clasif[df_col_correg[yy].iloc[num,0]] == clasif[df_col_correg[yy].iloc[num-1,0]]:
        if clasif[df_col_correg[yy].iloc[num-1,0]] == 2:
               fuente = df_col_correg[yy].iloc[num-1,0]
               categs[num-1] = [categoria,fuente,"-","-","-"]
        elif clasif[df_col_correg[yy].iloc[num-1,0]] == 3:
                subn1 = df_col_correg[yy].iloc[num-1,0]
                categs[num-1] = [categoria,fuente,subn1,"-","-"]
        elif clasif[df_col_correg[yy].iloc[num-1,0]] == 4:
                subn2 = df_col_correg[yy].iloc[num-1,0]
                categs[num-1] = [categoria,fuente,subn1,subn2,"-"]
        elif clasif[df_col_correg[yy].iloc[num-1,0]] == 5:
                subn3 = df_col_correg[yy].iloc[num-1,0]
                categs[num-1] = [categoria,fuente,subn1,subn2,subn3]

        num = num-1   

    return num

clasif_in_col = {}

for i in range(1990,2022):
    categs = {}

    for fila in range(1,168):   
        clasifactual = clasif[df_col_correg[i].iloc[fila,0]]
        if clasifactual == 1:
         categoria = df_col_correg[i].iloc[fila,0]
        elif clasifactual == 2:
         fuente = df_col_correg[i].iloc[fila,0]
        elif clasifactual == 3:
         subn1 = df_col_correg[i].iloc[fila,0]
        elif clasifactual == 4:
         subn2 = df_col_correg[i].iloc[fila,0]
        elif clasifactual == 5:
         subn3 = df_col_correg[i].iloc[fila,0]
    
        if clasifactual > clasif[df_col_correg[i].iloc[fila+1,0]]:
            if clasifactual == 2:
                categs[fila] = [categoria,fuente,"-","-","-"]
            if clasifactual == 3:
                categs[fila] = [categoria,fuente,subn1,"-","-"]
            if clasifactual == 4:
                categs[fila] = [categoria,fuente,subn1,subn2,"-"]
            if clasifactual == 5:
                categs[fila] = [categoria,fuente,subn1,subn2,subn3]
        
            cont = fila +1-1
            cont = whil_ant(i,cont,categoria, fuente, subn1, subn2, subn3)

            if clasif[df_col_correg[i].iloc[cont-1,0]] == clasif[df_col_correg[i].iloc[cont-2,0]]:
                cont = whil_ant(i,cont,categoria, fuente, subn1, subn2, subn3)
        
            if clasif[df_col_correg[i].iloc[cont-1,0]] == clasif[df_col_correg[i].iloc[cont-2,0]]:
                cont = whil_ant(i,cont,categoria, fuente, subn1, subn2, subn3)
        
            if clasif[df_col_correg[i].iloc[cont-1,0]] == clasif[df_col_correg[i].iloc[cont-2,0]]:
                cont = whil_ant(i,cont,categoria, fuente, subn1, subn2, subn3)

    categs[0] = [df_col_correg[i].iloc[0,0],"-","-","-","-"]
    categs[168] = [categoria,df_col_correg[i].iloc[168,0],"-","-","-"]

    keys_orden = sorted(categs.keys())

    categs_orden = {}
    for j in keys_orden:
        categs_orden[j] = categs[j]
    
    categss = pd.DataFrame.from_dict(categs_orden, orient = 'index')
    categss = categss.reset_index()
    categss.columns = ["Fila", "Categoria","Fuente","Subfuente1","Subfuente2","Subfuente3"]

    clasif_in_col[i] = categss.copy()


In [ ]:
#solo nivel 1

In [ ]:
nivel1 = {}

for i in range(1990,2022):
    nivel1[i]=df_col_correg[i].copy()

    for j in range(1,169):
        if clasif[df_col_correg[i].iloc[j,0]] != 1:
            nivel1[i] = nivel1[i].drop(j)

In [ ]:
nivel1[1990]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
46,[2] Procesos industriales y uso de productos,30144.818918,257.953577,872.91,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,32510.56772,NaN,0.000000
99,"[3] Agricultura, silvicultura y otros usos de ...",1564.992722,91145.253066,20911.772279,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113622.018066,NaN,3.093000
156,[4] Residuos,402.492892,10679.499256,3665.111658,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14747.103807,NaN,1.354876


In [ ]:
for i in range(1990,2022):
    nivel1[i] = nivel1[i].reset_index(drop=True)

In [ ]:
nivel1[1990]

,Fuentes,CO2,CH4,N2O,HFC-23,HFC-410A,HFC-43-10mee,HFC-125,HFC-134,HFC-134a,...,C2F6,C3F8,C4F6,c-C4F8,C5F8,NF3,SF6,EMISIONES_NETAS(Gg en CO2e),EMISIONES(sin 3B y 3D)(Gg en CO2e),Carbono_negro(Gg)
0,EMISIONES NETAS (Gg de CO2e),319999.845468,117840.420104,28337.973,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,467413.123797,466720.011075,77.557200
1,[1] Energía,287887.540936,15757.714204,2888.179063,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,306533.434203,NaN,73.109323
2,[2] Procesos industriales y uso de productos,30144.818918,257.953577,872.91,760.63584,0,0,0,0,0,...,92.197876,0,0,0,0,0,36.517125,32510.56772,NaN,0.000000
3,"[3] Agricultura, silvicultura y otros usos de ...",1564.992722,91145.253066,20911.772279,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113622.018066,NaN,3.093000
4,[4] Residuos,402.492892,10679.499256,3665.111658,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14747.103807,NaN,1.354876
